In [3]:
import pandas as pd
import dask.dataframe as dd

In [4]:
d = [
    {
        'id':[2],
        'object_names':['bab'],
        'paths':['/bab'],
        'size':[12],
        'type':['zip'],
        'upload':[True]
    },
    {
        'id':[1],
        'object_names':['bob'],
        'paths':['/bob'],
        'size':[2],
        'type':['zip'],
        'upload':[False]
    },
    {
        'id':[0],
        'object_names':['bib'],
        'paths':['/bib'],
        'size':[9],
        'type':['file'],
        'upload':[True]
        }
]

In [5]:
ddf = dd.from_pandas(pd.DataFrame([], columns=['id', 'object_names', 'paths', 'size', 'type', 'upload']))

In [6]:
for de in d:
    df = pd.DataFrame(de)
    if (len(ddf.index) == 0):
        ddf = dd.from_pandas(df, npartitions=1)
    else:
        ddf = dd.concat([ddf, dd.from_pandas(df, npartitions=1)])

In [7]:
ddf

,id,object_names,paths,size,type,upload
npartitions=3,,,,,,
,int64,string,string,int64,string,bool
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [8]:
del d[2]

In [9]:
d

[{'id': [2],
  'object_names': ['bab'],
  'paths': ['/bab'],
  'size': [12],
  'type': ['zip'],
  'upload': [True]},
 {'id': [1],
  'object_names': ['bob'],
  'paths': ['/bob'],
  'size': [2],
  'type': ['zip'],
  'upload': [False]}]

In [10]:
ddf.compute()

,id,object_names,paths,size,type,upload
0,2,bab,/bab,12,zip,True
0,1,bob,/bob,2,zip,False
0,0,bib,/bib,9,file,True


In [11]:
import os

In [12]:
path = os.path.abspath('../../../test_tree_large')

In [13]:
path

'/home/dave/test_tree_large'

In [14]:
df = pd.DataFrame()
folder_count = 0
file_count = 0
for folder, _, files in os.walk(path, topdown=True):
    if len(files) > 0:
        folder_count += 1
        file_count += len(files)
        # print(f'folder: {folder}')
        # print(f'sub_folders: {sub_folders}')
        # print(f'files: {files}')
        folder_files = [os.path.join(folder, file) for file in files]
        df = pd.concat([df, pd.DataFrame(folder_files, columns=['local_path'])], ignore_index=True)

In [15]:
df['size'] = df['local_path'].apply(lambda x: os.path.getsize(x))
df['object_names'] = df['local_path'].apply(lambda x: os.path.basename(x))
df['upload'] = False
df['type'] = 'file'

In [16]:
ddf = dd.from_pandas(df, npartitions=100)
ddf

,local_path,size,object_names,upload,type
npartitions=100,,,,,
0,string,int64,string,bool,string
91,...,...,...,...,...
...,...,...,...,...,...
8913,...,...,...,...,...
9002,...,...,...,...,...


In [17]:
current_objects = pd.Series(os.listdir('../../../test_tree_large/7/'))

In [18]:
current_objects = pd.DataFrame(current_objects, columns=['object_names'])
current_objects

,object_names
0,7_256.f
1,7_175.f
2,7_609.f
3,7_769.f
4,7_422.f
...,...
996,7_834.f
997,7_845.f
998,7_195.f
999,7_507.f


In [19]:
ddf['exists'] = ddf['object_names'].isin(current_objects['object_names'])

In [20]:
done = ddf.compute()

In [21]:
uploads = done[done['exists'] == False].drop(columns=['exists'])

In [22]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [23]:
max_zip_size = 1048576 * 100

In [24]:
uploads

,local_path,size,object_names,upload,type
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file
...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file


In [28]:
import numpy as np

In [ ]:

uploads['individual_upload'] = np.random.choice([True, False], size=len(uploads))
uploads

In [33]:
len(uploads)

8003

In [ ]:

cumulative_size = 0
batches = [None]
batch = [None]
batch_number = [None]
for row in uploads.iterrows():
    if row[1]['individual_upload']:
        batch_number.append(0)
    else:
        size = row[1]['size']
        if cumulative_size + size > max_zip_size:
            batches.append(1)
            batch = [1]
            cumulative_size = size
        else:
            batch.append(1)
            cumulative_size += size
            print(row[1].name)
            if row[1].name == uploads.index[-1]:
                print('inif')
                batches.append(1)
        batch_number.append(len(batches))
    print(f'row: {row[1].name} - batch: {len(batches)} - cumulative_size: {cumulative_size} - size: {size} - individual_upload: {row[1]["individual_upload"]}')
    # k+=1
    # if k == 10:
        # break

batch_number[-10:]

0
row: /home/dave/test_tree_large/test-test_tree_large-lsst-backup.csv - batch: 1 - cumulative_size: 96 - size: 96 - individual_upload: False
1
row: /home/dave/test_tree_large/test_tree_traversal.yaml - batch: 1 - cumulative_size: 372 - size: 276 - individual_upload: False
2
row: /home/dave/test_tree_large/test.csv - batch: 1 - cumulative_size: 334580 - size: 334208 - individual_upload: False
row: /home/dave/test_tree_large/3/3_425.f - batch: 1 - cumulative_size: 334580 - size: 334208 - individual_upload: True
4
row: /home/dave/test_tree_large/3/3_805.f - batch: 1 - cumulative_size: 1383156 - size: 1048576 - individual_upload: False
5
row: /home/dave/test_tree_large/3/3_795.f - batch: 1 - cumulative_size: 2431732 - size: 1048576 - individual_upload: False
row: /home/dave/test_tree_large/3/3_238.f - batch: 1 - cumulative_size: 2431732 - size: 1048576 - individual_upload: True
7
row: /home/dave/test_tree_large/3/3_562.f - batch: 1 - cumulative_size: 3480308 - size: 1048576 - individual_u

[0, 0, 40, 40, 40, 0, 40, 0, 40, 0]

In [52]:
len(batch_number)

8004

In [54]:
uploads['batch_number'] = batch_number[1:]
uploads

,local_path,size,object_names,upload,type,individual_upload,batch_number
0,/home/dave/test_tree_large/test-test_tree_larg...,96,test-test_tree_large-lsst-backup.csv,False,file,False,1
1,/home/dave/test_tree_large/test_tree_traversal...,276,test_tree_traversal.yaml,False,file,False,1
2,/home/dave/test_tree_large/test.csv,334208,test.csv,False,file,False,1
3,/home/dave/test_tree_large/3/3_425.f,1048576,3_425.f,False,file,True,0
4,/home/dave/test_tree_large/3/3_805.f,1048576,3_805.f,False,file,False,1
...,...,...,...,...,...,...,...
8998,/home/dave/test_tree_large/1/1_632.f,1048576,1_632.f,False,file,True,0
8999,/home/dave/test_tree_large/1/1_686.f,1048576,1_686.f,False,file,False,40
9000,/home/dave/test_tree_large/1/1_671.f,1048576,1_671.f,False,file,True,0
9001,/home/dave/test_tree_large/1/1_613.f,1048576,1_613.f,False,file,False,40


In [55]:
uploads[uploads['batch_number'] == 1]['size'].sum()

104143604

In [58]:
for bn in range(1, uploads['batch_number'].max()+1):
    size = uploads[uploads['batch_number'] == bn]['size'].sum()
    count = len(uploads[uploads['batch_number'] == bn])
    print(f'Batch Number: {bn}, Size: {size}, Count: {count}')
    if size > max_zip_size:
        print('Size is greater than max zip size')
    else:
        print('Size is less than max zip size')

Batch Number: 1, Size: 104143604, Count: 102
Size is less than max zip size
Batch Number: 2, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 3, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 4, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 5, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 6, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 7, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 8, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 9, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 10, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 11, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 12, Size: 104857600, Count: 100
Size is less than max zip size
Batch Number: 13, Size: 104857600, Count: 100
Size is less than max zip size
Batch Nu